<a href="https://colab.research.google.com/github/dotsnangles/from-keras-to-pytorch-and-more/blob/main/stylegan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개발 환경 구축

GPU의 정보를 확인합니다.

In [3]:
!nvidia-smi

Tue Jan  3 01:24:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

구글 드라이브를 코랩에 마운트합니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


개발 환경을 구축합니다.
- 아래 코드를 실행한 뒤 우측의 파일탭을 새로고침하면 /content/pyimgdata와 /content/stylegan2-ada-pytorch가 생성되어 있습니다.
- pyimgdata를 통해 훈련에 필요한 이미지를 크롤링하고 stylegan2-ada-pytorch의 스크립트를 사용해 훈련을 진행합니다.

In [5]:
!pip install -q torch==1.8.1 torchvision==0.9.1 flickrapi ninja
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!git clone https://github.com/jeffheaton/pyimgdata.git

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.
fatal: destination path 'pyimgdata' already exists and is not an empty directory.


# StyleGAN2 훈련

- 구글 드라이브와 코랩을 활용해 훈련을 진행합니다.
- 아래 구글 드라이브를 마운트하고 아래 폴더를 생성해줍니다.

    -  ./data/gan/images - RAW images I wish to train on.
    -  ./data/gan/datasets - Actual training datasets that I convert from the raw images.
    -  ./data/gan/experiments - The output from StyleGAN2, my image previews and saved network snapshots.
- 체크포인트를 구글 드라이브에 저장하며 진행하므로 코랩 연결 시간이 초과되어 연결이 끊어지더라도 후속 훈련을 수행할 수 있습니다.

In [ ]:
!mkdir -p /content/drive/MyDrive/data/gan/images
!mkdir -p /content/drive/MyDrive/data/gan/dataset
!mkdir -p /content/drive/MyDrive/data/gan/experiments 

# Flicker의 이미지를 크롤링하여 StyleGan2의 훈련 데이터로 사용합니다.

- pyimgdata를 활용하여 쉽게 Flicker의 이미지를 다운로드 할 수 있습니다.
- Flickr 계정을 생성하고 API Key와 Secret을 부여받아야 합니다.


다음 페이지에 접속해 API Key를 생성합니다.
- https://www.flickr.com/services/
- Key와 Secret을 각각 하단의 id와 secret에 대입합니다.

/content/pyimgdata/config_flickr.ini 설정이 필요합니다. 변경이 필요한 항목은 다음과 같습니다.
- id / secret / path / search / prefix

```
[FLICKR]
id = [your api id]
secret = [your secret/password]
[Download]
path = /content/drive/MyDrive/data/gan/images
search = christmas decorations
prefix = christmas
update_minutes = 1
license = 0,1,2,3,4,5,6,7,8,9,10
max_download = 100000
sources_file = sources.csv
[Process]
process = True
crop_square = True
min_width = 256
min_height = 256
scale_width = 256
scale_height = 256
image_format = jpg
```

아래 코드를 실행하면 크롤링이 진행됩니다.

In [ ]:
%cd /content/pyimgdata
!python flickr-download.py
%cd /content

# 훈련에 사용시 문제가 있는 이미지가 있는지 확인합니다.

- 모든 이미지는 같은 크기여야 하며 동일한 수의 컬러 채널을 가지고 있어야 합니다.
- 그렇지 않을시 훈련 도중 에러가 발생합니다.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


# 크롤링한 이미지를 활용해 데이터세트를 생성합니다.

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images --dest /content/drive/MyDrive/data/gan/dataset

# 초기 훈련을 시작합니다.

-  workers 설정이 적용되지 않는 경우 코드블록 실행을 중지 후 다시 시작합니다.
- 코드블록을 재실행했음에도 workers가 코랩의 cpu 코어 갯수인 2개로 설정되지 않는 경우 /content/stylegan2-ada-pytorch/train.py의 108번째 줄의 num_workers를 2로 수정한 뒤 다시 실행합니다.
- train.py를 살펴보면 더 많은 훈련 설정이 가능한 것을 살펴볼 수 있습니다.
- 40tik마다 성능지표인 fid 계산이 이루어집니다. fid가 5 이하인 경우 충분히 훈련이 진행된 것으로 판단할 수 있습니다.
- 코랩 연결이 종료되더라도 체크포인트부터 훈련을 재개할 수 있습니다.

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 10
WORKERS = 2

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --workers {WORKERS}"
!{cmd}

# 체크포인트 훈련 재개

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(EXPERIMENTS, "00012-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 10
WORKERS = 2

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA} --workers {WORKERS}"
!{cmd}

# 생성

In [14]:
import os

# Modify these to suit your needs

EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
RUN = "00014-dataset-auto1-resumecustom"
NETWORK = os.path.join(EXPERIMENTS, RUN, "network-snapshot-000080.pkl")
OUTDIR = os.path.join(EXPERIMENTS, RUN, 'out')
TRUNCATION = 0.7
SEEDS = '300-305'

# Build the command and run it

cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/generate.py --outdir {OUTDIR} --trunc {TRUNCATION} --seeds {SEEDS} --network {NETWORK}"
!{cmd}

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00014-dataset-auto1-resumecustom/network-snapshot-000080.pkl"...
Generating image for seed 300 (0/6) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 301 (1/6) ...
Generating image for seed 302 (2/6) ...
Generating image for seed 303 (3/6) ...
Generating image for seed 304 (4/6) ...
Generating image for seed 305 (5/6) ...
